In [1]:
# Import needed libraries and config.
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,a84a6e6f-1212-4d90-9647-0ee3f8b11bda
Bastion Host,bastion.fabric-testbed.net
Bastion Username,zeyang2_0033371586
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


In [4]:
# To simulate multisite communications, we randomly fetch 3 sites for node creation.
# Each site has one nodes.
slice_name = 'MySlice'
[site1,site2,site3] = fablib.get_random_sites(count=3)
print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

Sites: SEAT, STAR, INDI


In [7]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1, disk=100, image='default_ubuntu_20')
node1.add_fabnet()

# Node2
node2 = slice.add_node(name=node2_name, site=site2, disk=100, image='default_ubuntu_20')
node2.add_fabnet()

# Node3
node3 = slice.add_node(name=node3_name, site=site3, disk=100, image='default_ubuntu_20')
node3.add_fabnet()

#Submit Slice Request
slice.submit();


Retry: 11, Time: 287 sec


ID,dfb68c65-74a5-41d9-91b6-440fbc1ba868
Name,MySlice
Lease Expiration (UTC),2024-12-11 17:46:22 +0000
Lease Start (UTC),2024-12-10 17:46:22 +0000
Project ID,a84a6e6f-1212-4d90-9647-0ee3f8b11bda
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
0b871d47-961f-4530-b7e5-6c1db49fbad4,Node1,2,8,100,default_ubuntu_20,qcow2,seat-w2.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fecc:52ff,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fecc:52ff,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dcc03a62-dc83-4bc9-b8cf-7f6265d4316d,Node2,2,8,100,default_ubuntu_20,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe7a:a081,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe7a:a081,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b7edad01-8dc8-417b-9888-6054c9014cb2,Node3,2,8,100,default_ubuntu_20,qcow2,indi-w1.fabric-testbed.net,INDI,ubuntu,2001:18e8:fff0:3:f816:3eff:fee4:3065,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:fee4:3065,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
8ce5d1e0-a33f-4505-bb75-60f1a4ea5ea0,FABNET_IPv4_INDI,L3,FABNetv4,INDI,10.140.44.0/24,10.140.44.1,Active,
7cc51869-27ac-451f-8f4d-dfcb1e7a8e4f,FABNET_IPv4_SEAT,L3,FABNetv4,SEAT,10.139.4.0/24,10.139.4.1,Active,
8841ce9f-c4a0-4d65-9991-307095b74ca2,FABNET_IPv4_STAR,L3,FABNetv4,STAR,10.129.206.0/24,10.129.206.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-FABNET_IPv4_SEAT_nic-p1,p1,Node1,FABNET_IPv4_SEAT,100,auto,,0A:09:A9:2A:D0:23,enp7s0,enp7s0,10.139.4.2,4,HundredGigE0/0/0/7
Node2-FABNET_IPv4_STAR_nic-p1,p1,Node2,FABNET_IPv4_STAR,100,auto,,2E:C4:84:C0:9B:85,enp7s0,enp7s0,10.129.206.2,4,HundredGigE0/0/0/9
Node3-FABNET_IPv4_INDI_nic-p1,p1,Node3,FABNET_IPv4_INDI,100,auto,,4A:DB:4B:69:30:A3,enp7s0,enp7s0,10.140.44.2,6,HundredGigE0/0/0/5



Time to print interfaces 292 seconds


In [8]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           
node3 = slice.get_node(name=node3_name)

node1_addr = node1.get_interface(network_name=f'FABNET_IPv4_{node1.get_site()}').get_ip_addr()
node2_addr = node2.get_interface(network_name=f'FABNET_IPv4_{node2.get_site()}').get_ip_addr()
node3_addr = node3.get_interface(network_name=f'FABNET_IPv4_{node3.get_site()}').get_ip_addr()

print("node1_addr = " + str(node1_addr))
print("node2_addr = " + str(node2_addr))
print("node3_addr = " + str(node3_addr))

# Test code for Network connection
stdout, stderr = node2.execute(f'ping -c 5 {node1_addr}')
stdout, stderr = node3.execute(f'ping -c 5 {node1_addr}')

node1_addr = 10.139.4.2
node2_addr = 10.129.206.2
node3_addr = 10.140.44.2
PING 10.139.4.2 (10.139.4.2) 56(84) bytes of data.
64 bytes from 10.139.4.2: icmp_seq=1 ttl=61 time=48.0 ms
64 bytes from 10.139.4.2: icmp_seq=2 ttl=61 time=46.4 ms
64 bytes from 10.139.4.2: icmp_seq=3 ttl=61 time=46.4 ms
64 bytes from 10.139.4.2: icmp_seq=4 ttl=61 time=46.4 ms
64 bytes from 10.139.4.2: icmp_seq=5 ttl=61 time=46.5 ms

--- 10.139.4.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 46.439/46.754/47.970/0.607 ms
PING 10.139.4.2 (10.139.4.2) 56(84) bytes of data.
64 bytes from 10.139.4.2: icmp_seq=1 ttl=60 time=51.6 ms
64 bytes from 10.139.4.2: icmp_seq=2 ttl=60 time=51.0 ms
64 bytes from 10.139.4.2: icmp_seq=3 ttl=60 time=51.0 ms
64 bytes from 10.139.4.2: icmp_seq=4 ttl=60 time=51.0 ms
64 bytes from 10.139.4.2: icmp_seq=5 ttl=60 time=51.0 ms

--- 10.139.4.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rt

In [9]:
# Delele Slice after the experiment is done
slice.delete()